In [ ]:
import cv2
import numpy as np

# 加载图像
image_path = './111.png'
image = cv2.imread(image_path)

# 定义颜色范围 (使用 HSV 格式)
# 根据扫雷游戏常见颜色来设置范围，这些范围可以根据实际图像进一步调整
color_ranges = {
    '1': ((102, 159, 199), (122, 199, 255)),  # 将范围设为中心值的 ±10、±20 和 ±30
    '2': ((82, 165, 82), (102, 185, 102)),  # 绿色范围
}

# 初始化矩阵
num_rows, num_cols = 16, 30
matrix = [['H' for _ in range(num_cols)] for _ in range(num_rows)]

# 将图像转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义网格单元的大小
cell_height = hsv_image.shape[0] // num_rows
cell_width = hsv_image.shape[1] // num_cols

# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'
        
        # 检测每种颜色范围，并确定数字
        for num, (lower, upper) in color_ranges.items():
            # 创建掩膜来检测颜色范围内的像素
            mask = cv2.inRange(cell, np.array(lower), np.array(upper))
            # 如果检测到足够多的像素，则认为该单元格包含该数字
            if cv2.countNonZero(mask) > (cell_height * cell_width * 0.1):  # 20% 以上像素符合颜色范围
                detected_value = num
                break
        
        # 将检测到的值填入矩阵
        matrix[row][col] = detected_value


In [ ]:
# 打印矩阵，每行作为一行输出
for row in matrix:
    print(' '.join(row))


In [ ]:
# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 计算平均颜色值
        avg_color = cv2.mean(cell)[:3]  # 提取平均颜色的 HSV 值

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'
        
        # 检测每种颜色范围，并确定数字
        for num, (lower, upper) in color_ranges.items():
            # 创建掩膜来检测颜色范围内的像素
            mask = cv2.inRange(cell, np.array(lower), np.array(upper))
            # 如果检测到足够多的像素，则认为该单元格包含该数字
            if cv2.countNonZero(mask) > (cell_height * cell_width * 0.1):  # 20% 以上像素符合颜色范围
                detected_value = num
                break
        
        # 将检测到的值和平均颜色填入矩阵
        matrix[row][col] = (detected_value, avg_color)  # 存储数字和颜色值

# 打印矩阵及其颜色值
for row in matrix:
    print(row)


In [41]:
import cv2
import numpy as np
import time

# 加载图像
image_path = './111.png'
image = cv2.imread(image_path)

# 定义颜色范围 (使用 HSV 格式)
color_ranges = {
    'F': ((93, 47, 91), (109, 63, 107)),
    '0': ((97, 50, 57), (113, 66, 73)), 
    '1': ((97, 66, 82), (113, 82, 98)),  
    '2': ((81, 71, 83), (97, 87, 99)),   
    '3': ((118, 72, 95), (134, 88, 111)),
    '4': ((109, 69, 91), (125, 85, 107)),
    '5': ((68, 96, 87), (84, 112, 103)),


}

# 初始化矩阵
num_rows, num_cols = 16, 30
matrix = [['H' for _ in range(num_cols)] for _ in range(num_rows)]

# 排除的颜色范围 (BGR格式)
excluded_color = np.array([50, 58, 65])
lower_bound = excluded_color - 10  # 下界
upper_bound = excluded_color + 10    # 上界

# 将图像转换为 HSV 颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# print(hsv_image)


# 定义网格单元的大小
cell_height = hsv_image.shape[0] // num_rows
cell_width = hsv_image.shape[1] // num_cols

# 遍历每个网格单元并进行颜色检测
for row in range(num_rows):
    for col in range(num_cols):
        # 计算每个单元格的坐标
        x_start = col * cell_width
        y_start = row * cell_height
        x_end = x_start + cell_width
        y_end = y_start + cell_height

        # 截取每个单元格区域
        cell = hsv_image[y_start:y_end, x_start:x_end]

        # 创建掩膜以排除特定颜色范围
        mask = cv2.inRange(cell, lower_bound, upper_bound)

        # 计算平均颜色值，排除掩膜中的像素
        avg_color = cv2.mean(cell, mask=~mask)[:3]  # 提取平均颜色的 HSV 值

        # 初始化标记为隐藏 ('H')
        detected_value = 'H'

        # 根据平均颜色值决定矩阵的值
        for num, (lower, upper) in color_ranges.items():
            h_check = lower[0] <= avg_color[0] <= upper[0]
            s_check = lower[1] <= avg_color[1] <= upper[1]
            v_check = lower[2] <= avg_color[2] <= upper[2]
            
            #print(f"Checking {num}: H: {h_check}, S: {s_check}, V: {v_check}")

            if h_check and s_check and v_check:
                detected_value = num
                break

        # 将检测到的值和平均颜色填入矩阵
        # matrix[row][col] = (detected_value, avg_color)  # 存储数字和颜色值
        matrix[row][col] = detected_value

# 获取当前时间戳并格式化为文件名
timestamp = int(time.time())
file_name = f"{timestamp}.mine"

# 将输出保存到文件
with open(file_name, 'w') as file:
    file.write(f"{num_cols}x{num_rows}x99\n")
    for row in matrix:
        file.write("".join(row) + "\n")

print(f"输出已保存为 {file_name}")

# 打印矩阵的维度
print(f"{num_cols}x{num_rows}x99")

# # 打印矩阵
# for row in matrix:
#     print("".join(row))

    # 打印矩阵及其颜色值
for row in matrix:
    print(row)


输出已保存为 1730096210.mine
30x16x99
['0', '1', '1', '2', '1', '1', '0', '0', '1', 'H', 'H', 'H', 'H', 'H', '2', 'F', '2', '2', 'F', '1', '1', '1', '1', 'H', '0', '0', '0', '0', '0', '0']
['1', '2', 'F', '2', 'F', '1', '0', '0', '1', '2', 'H', 'H', 'H', 'H', '2', '2', 'F', '2', '1', '1', '1', 'F', 'H', '2', '2', '2', 'H', '0', '1', '1']
['H', '2', '1', '2', '1', '1', '0', '1', '1', '2', 'H', 'H', 'H', 'H', '1', '2', '2', '4', '4', 'H', '2', '2', '2', 'H', 'F', 'H', '1', '1', 'H', '0']
['2', '2', '1', '0', '0', '0', '0', '2', 'F', '3', 'H', 'H', 'H', 'H', '1', '1', 'F', '2', 'F', 'H', 'H', '1', '2', '1', '3', '3', '4', 'H', 'H', '2']
['1', 'H', '1', '0', '0', '0', '0', '2', 'H', 'H', 'H', 'H', 'H', 'H', '4', '3', '3', '3', '4', 'H', '2', 'H', '1', '1', 'F', '1', 'F', 'H', '2', '1']
['1', '1', '1', '0', '0', '1', '2', '3', 'H', 'H', 'H', 'H', 'H', 'H', '1', 'H', 'H', '2', '1', '1', 'H', '2', '2', '2', '1', '1', '4', '2', '1', '1']
['0', '0', '1', '1', '1', '1', 'F', 'H', '4', 'H', 'H', 'H', '